- 소매점 판매 데이터의 고객에 대한 분석을 수행하고자 한다.
1. 03_Merge_Data 데이터를 이용해, '총구매금액'이 높은 상위 100명의 고객 명단을 추출하여, result_member_vip.xlsx로 저장 하시오.
2. 회원 '성별'에 따른 '총구매금액'의 합을 비교하고자 한다. '성별' 별 '총구매금액'의 합을 Table로 계산하시오.
3. '결혼유무'에서 '기혼'에 해당하는 고객들 중 '총방문횟수'가 높은 상위 30명의 고객명단을 확인 하시오.
4. '연령'을 이용해, 고객 '연령대'를 계산하고, '총구매금액'의 합이 가장 높은 2개의 '연령대'를 확인하여, 두 연령대 간의 '총구매금액'의 평균의 차이를 계산 하시오.
5. '회원상태'에 따른 '총구매금액'의 평균을 시각화 하시오.

In [1]:
import pandas as pd
import plotly.express as px

In [4]:
df1 = pd.read_csv('03_Merge_Data.csv',index_col=0)
pv1 = df1.pivot_table(index='회원상태',values='총구매금액',aggfunc='mean').reset_index()
px.histogram(pv1,x='회원상태',y='총구매금액')

6. '총구매금액'을 '총방문횟수'로 나누어, '1회방문구매평균'을 계산하고, 생성된 파생변수를 바탕으로 아래와 같이 고객등급을 분류하시오.
    - '1회방문구매평균'이 10만원 이상 고객은 A / 5만원 이상은 B / 2만원 이상 C / 나머지는 D 등급으로 분류
    - 분류된 고객 등급은 '고객등급'이라는 새로운 파생변수로 생성

In [6]:
df1['1회방문구매평균'] = df1['총구매금액'] / df1['총방문횟수']
def cus_grade(row) :
    if row >= 100000 :
        return 'A'
    elif row >= 50000 :
        return 'B'
    elif row >= 20000 :
        return 'C'
    else : return 'D'
df1['고객등급'] = df1['1회방문구매평균'].apply(cus_grade)

7. 새로 생성된 파생변수 '고객등급'에서 '기혼자가 가장 많은 그룹은 어디 그룹인지 확인하시오.

In [19]:
df1['label'] = 1
pv2 = df1.pivot_table(index=['고객등급','결혼유무'],values='label',aggfunc='sum').reset_index()
pv2.loc[pv2['결혼유무'] == '기혼'].sort_values(by='label',ascending=False).head(1)['고객등급']

4    C
Name: 고객등급, dtype: object

8. df2 데이터에서 '주소'데이터를 바탕으로, '지역(시)'이라는 이름의 새로운 파생변수를 생성 하시오.
    - 서울시 / 용인시 / 성남시 /수원시 /광주시 / 나머지 지역은 기타로 분류

In [28]:
def new_address(row) :
    if row[0:2] == '서울' :
        return '서울시'
    elif row[3:6] in ['용인시','성남시','수원시','광주시'] :
        return row[3:6]
    else : return '기타'

df1['지역(시)'] = df1['주소'].apply(new_address)
df1['지역(시)'].unique()

array(['서울시', '용인시', '성남시', '기타', '수원시', '광주시'], dtype=object)

9. '고객등급'을 입력했을 때, 해당 고객등급에서 '성별'에 따른 '총구매금액' 기술통계량을 계산하여 'result9_등급명.xlsx'저장하는 프로그램을 구성 하시오.

In [33]:
def input_grade() :
    grade = input()
    df2 = df1.loc[df1['고객등급']==grade]
    pv = df2.pivot_table(index='성별',values='총구매금액',aggfunc=['sum','min','median','max','mean']).reset_index()
    pv.to_excel(f'result9_{grade}.xlsx')
    return
input_grade()

10. 각 회원 별 ‘물품대분류’에서 각 물품을 어느 정도 비중으로 구매했는지 계산하고자 한다. df_product 에서, ‘물품대분류’의 항목을 확인하고, ‘회원번호’ 별 ‘물품 대분류의 구매수량을 계산한 후, 한명의 회원이 구매한 전체 수량으로 나누어 각 품목 별 구매 비율을 계산 df1에 병합하시오.

In [138]:
df_member = pd.read_csv('03_store_member.csv')
df_product = pd.read_csv('03_store_product.csv')

In [139]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735783 entries, 0 to 735782
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   공급일자      735783 non-null  int64  
 1   물품코드      735783 non-null  int64  
 2   물품대분류     735783 non-null  object 
 3   물품중분류     735783 non-null  object 
 4   물품소분류     735783 non-null  object 
 5   물품명       735783 non-null  object 
 6   구매수량      735783 non-null  float64
 7   구매금액      735783 non-null  int64  
 8   구매매장      735783 non-null  object 
 9   반품_원거래일자  10337 non-null   float64
 10  회원번호      735761 non-null  float64
dtypes: float64(3), int64(3), object(5)
memory usage: 61.7+ MB


In [140]:
def new_cus_num(row) :
    return str(row)[:4]
df_product['회원번호'] = df_product['회원번호'].apply(new_cus_num)

In [141]:
df_product['label'] = 1
pv3 = df_product.pivot_table(index='회원번호',columns='물품대분류',values='label',aggfunc='sum').fillna(0)

In [152]:
df_10 = pd.DataFrame(columns=['회원번호','간식', '건강', '과실', '급식용', '기타', '반찬', '베이커리', '생활용품', '서류', '수산',
                            '양념/가루', '음료', '잡곡', '주곡', '주류', '채소', '축산물'])

df_10['회원번호'] = pv3.index

In [157]:
for i in range(len(pv3)) :
    summ = pv3.iloc[i].sum()
    for j in range(len(pv3.columns)) :
        a = pv3.columns[j]
        b = str(round((pv3.iloc[i][j] / summ) * 100, 2))      
        df_10.at[i,a]= b+'%'

C:\Users\BigData1\AppData\Local\Temp\ipykernel_13424\297214490.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [159]:
df_10['총합'] = '100%'
df_10.head(10)

,회원번호,간식,건강,과실,급식용,기타,반찬,베이커리,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,총합
0,2101,7.89%,2.34%,0.88%,0.0%,0.0%,15.5%,0.0%,2.05%,1.75%,1.17%,3.22%,1.46%,1.46%,0.58%,0.0%,35.67%,26.02%,100%
1,2102,27.63%,0.0%,7.09%,0.0%,0.0%,10.76%,0.0%,0.98%,2.69%,4.4%,3.67%,1.71%,1.22%,3.91%,0.0%,24.94%,11.0%,100%
2,2103,16.08%,1.94%,3.13%,0.0%,0.0%,11.7%,0.0%,3.32%,1.31%,9.76%,3.69%,3.75%,0.88%,0.75%,0.19%,29.22%,14.27%,100%
3,2104,14.07%,0.34%,5.08%,0.0%,0.0%,12.8%,0.0%,2.37%,1.44%,7.37%,4.75%,2.46%,1.1%,1.36%,0.08%,31.36%,15.42%,100%
4,2105,14.26%,0.59%,5.27%,0.0%,0.0%,10.74%,0.0%,1.76%,2.93%,4.88%,4.49%,4.69%,2.93%,1.17%,0.0%,30.66%,15.62%,100%
5,2106,6.74%,0.33%,4.77%,0.0%,0.0%,7.57%,0.0%,0.0%,1.81%,4.93%,5.76%,0.99%,1.97%,3.45%,0.0%,45.89%,15.79%,100%
6,2107,11.06%,0.17%,3.94%,0.0%,0.0%,6.7%,0.0%,3.43%,0.67%,8.96%,4.19%,1.76%,2.35%,1.34%,0.0%,43.38%,12.06%,100%
7,2108,15.51%,0.34%,4.56%,0.0%,0.0%,9.35%,0.0%,0.23%,1.03%,5.93%,3.65%,2.05%,1.14%,0.46%,0.0%,36.03%,19.73%,100%
8,2109,11.95%,1.03%,8.05%,0.0%,0.0%,6.09%,0.0%,3.33%,1.61%,5.29%,4.02%,3.33%,0.92%,1.61%,0.0%,41.49%,11.26%,100%
9,2110,20.67%,1.21%,3.11%,0.0%,0.0%,9.08%,0.0%,2.85%,1.3%,8.22%,1.3%,4.76%,0.69%,1.64%,0.09%,25.43%,19.64%,100%


In [ ]:
######## 정답코드
for i in range(len(pv3)) :
    s = pv3.iloc[i]
    summ = s.sum()
    for j in range(len(s)) :
        s[j] = round(((s[j]/summ) * 100),2)
    pv3.iloc[i] = s
pv3.head(10)

In [165]:
df1['회원번호'] = df1['회원번호'].astype(str)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4285 entries, 0 to 4284
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회원번호      4285 non-null   object 
 1   회원상태      4285 non-null   object 
 2   성별        4285 non-null   object 
 3   결혼유무      3360 non-null   object 
 4   주소        4285 non-null   object 
 5   연령        4285 non-null   int64  
 6   총구매금액     4285 non-null   int64  
 7   총구매수량     4285 non-null   float64
 8   총방문횟수     4285 non-null   int64  
 9   1회방문구매평균  4285 non-null   float64
 10  고객등급      4285 non-null   object 
 11  label     4285 non-null   int64  
 12  지역(시)     4285 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 597.7+ KB


In [168]:
df2 = pd.merge(df1,df_10,on='회원번호')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4285 entries, 0 to 4284
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회원번호      4285 non-null   object 
 1   회원상태      4285 non-null   object 
 2   성별        4285 non-null   object 
 3   결혼유무      3360 non-null   object 
 4   주소        4285 non-null   object 
 5   연령        4285 non-null   int64  
 6   총구매금액     4285 non-null   int64  
 7   총구매수량     4285 non-null   float64
 8   총방문횟수     4285 non-null   int64  
 9   1회방문구매평균  4285 non-null   float64
 10  고객등급      4285 non-null   object 
 11  label     4285 non-null   int64  
 12  지역(시)     4285 non-null   object 
 13  간식        4285 non-null   object 
 14  건강        4285 non-null   object 
 15  과실        4285 non-null   object 
 16  급식용       4285 non-null   object 
 17  기타        4285 non-null   object 
 18  반찬        4285 non-null   object 
 19  베이커리      4285 non-null   object 
 20  생활용품      4285 non-null   obje

11. df_product데이터의 구조와 타입을 확인 하시오.

In [160]:
print(df_product.shape)
df_product.info()

(735783, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735783 entries, 0 to 735782
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   공급일자      735783 non-null  int64  
 1   물품코드      735783 non-null  int64  
 2   물품대분류     735783 non-null  object 
 3   물품중분류     735783 non-null  object 
 4   물품소분류     735783 non-null  object 
 5   물품명       735783 non-null  object 
 6   구매수량      735783 non-null  float64
 7   구매금액      735783 non-null  int64  
 8   구매매장      735783 non-null  object 
 9   반품_원거래일자  10337 non-null   float64
 10  회원번호      735783 non-null  object 
 11  label     735783 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 67.4+ MB


12. df_product데이터의'물품대분류' 중 가장 많이 판매되는(수량)3가지 항목을 추출하여, df_product_top이라는 변수로 선언하고, df_product_top에서 '구매매장'별 '구매금액'의 합과 평균을 계산 하시오.